<a href="https://colab.research.google.com/github/munakaghamelu/comp0031_photoToSketch/blob/main/photoToSketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
np.random.seed(0)
import torch
torch.manual_seed(0)
import random
random.seed(0)
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.autograd import Variable

from torchvision.utils import save_image
import cv2
import os
import time
import copy
import torchvision
import torchvision.models as models
import matplotlib.pyplot as plt

In [ ]:
image_size = 100

class AutoEncoder(Module):
    # IMPLEMENT THIS AUTO-ENCODER CLASS
    def __init__(self,bottleneck_size):
        super(AutoEncoder, self).__init__()
        # INITIALIZE YOUR TRAINING PARAMETERS HERE.
        input_shape = image_size**2*3
        self.enc_h1 = nn.Linear(in_features=input_shape, out_features=50)
        self.enc_out = nn.Linear(in_features=50, out_features=bottleneck_size)

        self.enc_mu = nn.Linear(2*50, Nz)
        self.enc_sigma = nn.Linear(2*50, Nz)
        
        self.dec_h1 = nn.Linear(in_features=Nz, 2*dec_hidden_size)
        self.lstm = nn.LSTM(Nz+5, dec_hidden_size, dropout=dropout)
        self.dec_params = nn.Linear(dec_hidden_size, 6*M+3)

        # self.dec_h1 = nn.Linear(in_features=bottleneck_size, out_features=50)
        # self.dec_out = nn.Linear(in_features=50, out_features=input_shape)
        
    def encoder(self,image):
        # WRITE ENCODER ARCHITECTURE HERE
        h1_ac = torch.relu(self.enc_h1(image))
        mu = self.enc_mu(h1_ac)
        logVar = self.sigma(h1_ac)
        std = logVar.mul(0.5).exp()
        z_size = mu.size()
        N = torch.normal(torch.zeros(z_size), torch.ones(z_size))
        z = mu + std*N
        # mu and logVar are need for LKL, this is because its a variational autoencoder model
        return z, mu, logVar
    
    def decoder(self, concatenated_vector, z, hidden_cell=None):
        # WRITING DECODER FOR SKETCH-RNN
        if hidden_cell is None:
          hidden, cell = torch.split(F.tanh(self.dec_h1(z)),dec_hidden_size,1)
          hidden_cell = (hidden.unsqueeze(0).contiguous(), cell.unsqueeze(0).contiguous())
          outputs,(hidden, cell) = self.lstm(concatenated_vector, hidden_cell)
        if self.training:
            y = self.dec_params(outputs.view(-1, dec_hidden_size))
        else:
            y = self.dec_params(hidden.view(-1, dec_hidden_size))
        # separate pen and mixture params:
        params = torch.split(y,6,1)
        params_mixture = torch.stack(params[:-1]) # trajectory
        params_pen = params[-1] #pen up/down
        # identify mixture params:
        pi,mu_x,mu_y,sigma_x,sigma_y,rho_xy = torch.split(params_mixture,1,2)
        # preprocess params:
        if self.training:
            len_out = Nmax+1
        else:
            len_out = 1
                                   
        pi = F.softmax(pi.transpose(0,1).squeeze()).view(len_out,-1,M)
        sigma_x = torch.exp(sigma_x.transpose(0,1).squeeze()).view(len_out,-1,M)
        sigma_y = torch.exp(sigma_y.transpose(0,1).squeeze()).view(len_out,-1,M)
        rho_xy = torch.tanh(rho_xy.transpose(0,1).squeeze()).view(len_out,-1,M)
        mu_x = mu_x.transpose(0,1).squeeze().contiguous().view(len_out,-1,M)
        mu_y = mu_y.transpose(0,1).squeeze().contiguous().view(len_out,-1,M)
        q = F.softmax(params_pen).view(len_out,-1,3)
        return pi,mu_x,mu_y,sigma_x,sigma_y,rho_xy,q,hidden,cell
    
    def forward(self,image):
        # PUT IT TOGETHER HERE
        # Get the inputs ready for the decoder part
        #batch, lengths = make_batch(batch_size)
        z, mu, sigma = self.encoder(image)
        # could work on changing the position 1 is placed
        sos = torch.stack([torch.Tensor([0,0,1,0,0])]*batch_size).unsqueeze(0)
        batch_init = torch.cat([sos, sos],0)
        z_stack = torch.stack([z]*(Nmax+1))
        inputs = torch.cat([batch_init, z_stack],2)
        self.pi, self.mu_x, self.mu_y, self.sigma_x, self.sigma_y, self.rho_xy, self.q, _, _ = self.decoder(inputs, z)
        #decoded_image = self.decoder(code)
        return decoded_image

In [ ]:
class Chairs(Dataset):
    # IMPLEMENT THIS DATA LOADING CLASS
    def __init__(self, dataset_path=""):
        # DEFINE YOUR PARAMETERS AND VARIABLES YOU NEED HERE.
        self.transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])
        self.input_imgs_names = sorted(os.listdir(dataset_path))
        self.input_imgs_paths = [os.path.join(dataset_path, name) for name in self.input_imgs_names]
        
    def __len__(self):
        # RETURN SIZE OF DATASET
        length = len(self.input_imgs_names)
        return length

    # cv2 transforms data into tensors
    def __getitem__(self, idx):
        # RETURN IMAGE AT GIVEN idx
        # use cv2 to load images
        img = cv2.imread(self.input_imgs_paths[idx])
        input_image = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        input_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        input_image = cv2.resize(input_image, (image_size, image_size), interpolation=cv2.INTER_NEAREST)
        input_image = self.transform(input_image)
        return input_image

In [ ]:
# Hyperparameters

dec_image_size = 512
Nz = 128
M = 20
batch_size = 20 #encoder code uses 100
dropout = 0.9

chairs_train_data_path = "/content/drive/MyDrive/sketch-rnn/Datasets/quickdraw/chair/train"
chairs_train_dataset = Chairs(chairs_train_data_path)
torch_train_chairs = DataLoader(chairs_train_dataset, shuffle=True, batch_size=batch_size, num_workers=1)

chairs_val_data_path = "/content/drive/MyDrive/sketch-rnn/Datasets/quickdraw/chair/val"
chairs_val_dataset = Chairs(chairs_val_data_path)
torch_val_chairs = DataLoader(chairs_val_dataset, shuffle=True,batch_size=batch_size,num_workers=1)


In [ ]:
def reconstruction_loss(producedSketch, desiredSketch):
    # L2 RECONSTRUCTION LOSS
    criterion = nn.MSELoss()
    return criterion(producedSketch, desiredSketch)

epochs = 100 # CHOOSE YOUR EPOCH SIZE TO GET BEST RESULTS

# May not need this if using pytorch model
chairs_bottleneck_size = 25 # CHOOSE YOUR BOTTLENECK SIZE. 

model_chairs = AutoEncoder(chairs_bottleneck_size)

# Optimizer section, using Adam
chairs_optimizer = optim.Adam(model_chairs.parameters(), lr=1e-3) 

train_loss = []
for ep in range(epochs):
    running_loss = 0.0
    for i,data in enumerate(torch_train_chairs):
        # This is dependent on your pytorch model specification

        # reshape mini-batch data to [N, 784] matrix so it can be loaded
        data = data.view(-1, data.shape[1]*data.shape[2]*data.shape[3])
        
        # resetting the gradients back to zero, pytorch accumulates gradients each backward pass
        chairs_optimizer.zero_grad()
        
        # encoder layer pass, compute the reconstructions
        new_img = model_chairs(data)
        
        # compute loss to optimize reconstruction with
        loss = reconstruction_loss(new_img, data)
        
        # back propagation
        loss.backward()
        
        # updtae parameters
        chairs_optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        running_loss += loss.item()
    
    loss = running_loss / 100 # 100 images
    
    #train_loss.append(loss)
    print(f'Epoch {ep+1} of {epochs}, Train Loss: {loss:.5f}')